In [ ]:
import cv2
from ultralytics import YOLO
import torch
import time
import math
from collections import defaultdict

# --- Main Settings ---
WEBCAM_INDEX = 0    # Default webcam is 0
# CONFIDENCE_THRESHOLD = 0.5 # Initial confidence threshold
WINDOW_NAME = "YOLO Real-Time Object Detection"
FONT = cv2.FONT_HERSHEY_SIMPLEX
FONT_SCALE = 0.7
FONT_COLOR = (255, 255, 255)  # White
BOX_COLOR = (0, 255, 0)      # Green
BOX_THICKNESS = 2
COUNT_COLOR = (0, 255, 255)  # Yellow for the count text

# Dummy callback function for the trackbar (required by OpenCV)
def on_trackbar(val):
    pass

def main():
    # GPU VERIFICATION
    if torch.cuda.is_available():
        device = torch.device("cuda")
        print(f"GPU is available. Using device: {torch.cuda.get_device_name(0)}")
    else:
        device = torch.device("cpu")
        print("GPU not available. Using CPU.")

    # 1. Load the pretrained YOLOv8 model and move it to the selected device
    print("Loading model...")
    # The model will be automatically sent to the 'device' (GPU or CPU)
    model = YOLO("yolov8n.pt") # Using the nano version for complete performance
    model.to(device) # Explicitly send the model to the GPU
    class_names = model.names
    print("Model loaded successfully.")

    # 2. Initialize webcam capture
    cap = cv2.VideoCapture(WEBCAM_INDEX)
    if not cap.isOpened():
        print("Error: Could not open webcam.")
        return

    # BONUS FEATURE SETUP: Interactive Confidence Trackbar
    cv2.namedWindow(WINDOW_NAME)
    # The trackbar value is an integer from 0 to 100. We'll divide by 100.
    initial_confidence = 50 # Corresponds to 0.5
    cv2.createTrackbar("Confidence", WINDOW_NAME, initial_confidence, 100, on_trackbar)

    # Variables for FPS calculation
    prev_frame_time = 0

    while True:
        # 3. Read a frame from the webcam
        success, frame = cap.read()
        if not success:
            print("Error: Failed to capture frame.")
            break

        # Get the current confidence threshold from the trackbar
        confidence_threshold = cv2.getTrackbarPos("Confidence", WINDOW_NAME) / 100.0

        # BONUS FEATURE SETUP: Object Counter
        # Initialize a dictionary to store the count of each object class for the current frame
        object_counts = defaultdict(int)

        # 4. Perform inference on the frame
        # The model will process the frame on the GPU
        results = model(frame, stream=True, verbose=False)

        # 5. Process and display detection results
        for r in results:
            boxes = r.boxes
            for box in boxes:
                confidence = box.conf[0]
                # Filter detections by the real-time confidence threshold
                if confidence < confidence_threshold:
                    continue

                # Get class name
                cls_index = int(box.cls[0])
                cls_name = class_names[cls_index]

                # Increment the count for the detected class
                object_counts[cls_name] += 1

                # Get bounding box coordinates and draw them
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                label = f"{cls_name}: {confidence:.2f}"
                cv2.rectangle(frame, (x1, y1), (x2, y2), BOX_COLOR, BOX_THICKNESS)
                cv2.putText(frame, label, (x1, y1 - 10), FONT, FONT_SCALE, FONT_COLOR, BOX_THICKNESS)

        # 6. Calculate and display FPS
        new_frame_time = time.time()
        if prev_frame_time > 0:
            fps = 1 / (new_frame_time - prev_frame_time)
            cv2.putText(frame, f"FPS: {int(fps)}", (10, 30), FONT, FONT_SCALE, (0, 0, 255), BOX_THICKNESS)
        prev_frame_time = new_frame_time

        # BONUS FEATURE DISPLAY: Object Counts
        # Display the counts on the top-right of the screen
        y_offset = 40
        # Sort items for a consistent display order
        for obj_name, count in sorted(object_counts.items()):
            count_text = f"{obj_name}: {count}"
            cv2.putText(frame, count_text, (frame.shape[1] - 180, y_offset), FONT, FONT_SCALE, COUNT_COLOR, BOX_THICKNESS)
            y_offset += 30 # Move down for the next line

        # 7. Display the final frame in the named window
        cv2.imshow(WINDOW_NAME, frame)

        # 8. Exit loop on 'q' key press
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # 9. Release resources
    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()